# Task 1 : Data Collection

In [79]:
import requests
import shutil
from datetime import date,timedelta
import calendar
from zipfile import ZipFile
import pandas as pd
from pathlib import Path 
import urllib.request
import os

In [80]:
# Creating variables for storing directory paths
DATASET_PATH = Path("dataset") # This folder  stores raw data downloaded from NSE website
DATASET_CLEANED_PATH = Path("dataset_cleaned") # This folder  stores cleaned data (EQ series)
HOLIDAY_PATH=Path("holiday_list") # This folder  stores holidays list of NSE
DB_SCRIPTS_PATH=Path("DB_Scripts") # This follder stores sql scripts

In [81]:
# Delete the directories if exists
if os.path.exists(DATASET_PATH) and os.path.isdir(DATASET_PATH):
    shutil.rmtree(DATASET_PATH)
if os.path.exists(DATASET_CLEANED_PATH) and os.path.isdir(DATASET_CLEANED_PATH):
    shutil.rmtree(DATASET_CLEANED_PATH)

In [82]:
# Create new directories if it doesnt exist
DATASET_PATH.mkdir(parents=True, exist_ok=True)
DATASET_CLEANED_PATH.mkdir(parents=True, exist_ok=True)
HOLIDAY_PATH.mkdir(parents=True, exist_ok=True)
DB_SCRIPTS_PATH.mkdir(parents=True, exist_ok=True)

In [83]:
# https://archives.nseindia.com/content/historical/EQUITIES/2023/SEP/cm21SEP2023bhav.csv.zip
# https://archives.nseindia.com/content/historical/EQUITIES/2023/SEP/cm22SEP2023bhav.csv.zip
# https://strftime.org/

'''
Function to download files from a url
'''
def download_file(url):
    local_filename = url.split('/')[-1] # retrieve the URL part after https://
    with requests.get(url, stream=True) as r:
        print("Inside the download_file function")
        DATASET_FILE_NAME=DATASET_PATH / local_filename # Setting the file path for file to be downloaded
        with open(DATASET_FILE_NAME, 'wb') as f:
            shutil.copyfileobj(r.raw, f) # copying the downloaded stream to local file
    return local_filename

In [84]:
#current_date=date.today()
#reference_date=date(2023,11,3)
#reference_date=date(2023,10,27) #Oct 24,2023 is holiday
#reference_date=date(2023,11,3)
reference_date=date(2023,11,16) 

number_of_days=5 # number of days of file to be downloaded w.r.t reference date
base_url="https://archives.nseindia.com/content/historical/EQUITIES/2023/#/cm{}bhav.csv.zip"

i=0 # to check number of successful downloads which shall be equal of number of dasy after successfully download
j=0 # to check number of days skipped

In [85]:
HOLIDAY_FILE_NAME=HOLIDAY_PATH / "holiday_list_nse.csv"

In [86]:
# Extracting holiday list of NSE
df=pd.read_csv(HOLIDAY_FILE_NAME) # Reading the csv file which contains the holiday list
df=df[df['YEAR']==reference_date.year] # Filtering the data for year of reference date
df['DATE'] = pd.to_datetime(df['DATE']) # converting the DATE column into datetime object since by default panda stores it as object data type
#df['DATE'] = df['DATE'].dt.date
df.head()

C:\Users\Shweta\AppData\Local\Temp\ipykernel_7232\1531252967.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DATE'] = pd.to_datetime(df['DATE']) # converting the DATE column into datetime object since by default panda stores it as object data type


,YEAR,DATE,DAY,DESCRIPTION
0,2023,2023-01-26,Thursday,Republic Day
1,2023,2023-03-07,Tuesday,Holi
2,2023,2023-03-30,Thursday,Ram Navami
3,2023,2023-04-04,Tuesday,Mahavir Jayanti
4,2023,2023-04-07,Friday,Good Friday


In [87]:
holiday_dates=df['DATE'].dt.date.to_list() # converting the DATE column into list data type containing dates
holiday_dates

[datetime.date(2023, 1, 26),
 datetime.date(2023, 3, 7),
 datetime.date(2023, 3, 30),
 datetime.date(2023, 4, 4),
 datetime.date(2023, 4, 7),
 datetime.date(2023, 4, 14),
 datetime.date(2023, 5, 1),
 datetime.date(2023, 6, 29),
 datetime.date(2023, 8, 15),
 datetime.date(2023, 9, 19),
 datetime.date(2023, 10, 2),
 datetime.date(2023, 10, 24),
 datetime.date(2023, 11, 14),
 datetime.date(2023, 11, 27),
 datetime.date(2023, 12, 25)]

In [88]:
temp=reference_date

while i<number_of_days:  # to repeat the loop until the files are downloaded number_of_days times
    if calendar.day_name[temp.weekday()] in 'Saturday':  # skip the date if its saturday
        print("{} is weekend (Saturday)".format(temp.strftime("%d%b%Y").upper()))
        temp=temp+timedelta(days=-1) # decrease the temp by 1 day as we are going backwards from reference date
    elif calendar.day_name[temp.weekday()] in 'Sunday':   # skip the date if its sunday     
        print("{} is weekend (Sunday)".format(temp.strftime("%d%b%Y").upper()))
        temp=temp+timedelta(days=-1) # decrease the temp by 1 day as we are going backwards from reference date
    elif temp in holiday_dates:    # skip the date if its holiday as per NSE   
        print("{} is holiday".format(temp.strftime("%d%b%Y").upper()))
        temp=temp+timedelta(days=-1) # decrease the temp by 1 day as we are going backwards from reference date
    else:
        url_for_csv=base_url.replace('{}',temp.strftime("%d%b%Y").upper())
        url_for_csv=url_for_csv.replace('#',temp.strftime("%b").upper())
        print(url_for_csv)
        download_file(url_for_csv)
        i=i+1    
        print("Downloaded file for  {}".format(temp.strftime("%d%b%Y").upper()))
        DATASET_FILE_NAME=DATASET_PATH / url_for_csv.split('/')[-1]
        with ZipFile(DATASET_FILE_NAME, 'r') as f:
            #extract in current directory
            f.extractall(DATASET_PATH)
        temp=temp+timedelta(days=-1)
   

https://archives.nseindia.com/content/historical/EQUITIES/2023/NOV/cm16NOV2023bhav.csv.zip
Inside the download_file function
Downloaded file for  16NOV2023
https://archives.nseindia.com/content/historical/EQUITIES/2023/NOV/cm15NOV2023bhav.csv.zip
Inside the download_file function
Downloaded file for  15NOV2023
14NOV2023 is holiday
https://archives.nseindia.com/content/historical/EQUITIES/2023/NOV/cm13NOV2023bhav.csv.zip
Inside the download_file function
Downloaded file for  13NOV2023
12NOV2023 is weekend (Sunday)
11NOV2023 is weekend (Saturday)
https://archives.nseindia.com/content/historical/EQUITIES/2023/NOV/cm10NOV2023bhav.csv.zip
Inside the download_file function
Downloaded file for  10NOV2023
https://archives.nseindia.com/content/historical/EQUITIES/2023/NOV/cm09NOV2023bhav.csv.zip
Inside the download_file function
Downloaded file for  09NOV2023


# Task 2 : Data Cleaning

In [89]:
import pandas as pd 
import os 
import glob 
from datetime import datetime

In [90]:
# use glob to get all the csv files  
# in the folder 
#path = os.getcwd() 
csv_files = glob.glob(os.path.join(DATASET_PATH, "*.csv")) 

In [91]:
  
# loop over the list of csv files 
for f in csv_files: 
      
    # read the csv file 
    df = pd.read_csv(f) 
      
    # print the location and filename 
    print('Location:', f) 
    file_name= f.split("\\")[-1]
    print('File Name:', file_name) 
    
    # retrieving the date stored between 'cm' and 'bhav' substring. sample 'cm23OCT2023bhav', to be retrieved 23OCT2023
    sub1 = "cm"
    sub2 = "bhav"
 
    file_name=file_name.replace(sub1,"*") # substituting 'cm' in file_name with '*
    file_name=file_name.replace(sub2,"*") # substiting 'bhav' in file_name with '*'
    re=file_name.split("*") # splitting the file_name with '*'. This will give us date at index=1 location
    res=re[1] # storing the date

    date_str=res
    date_format = '%d%b%Y' # setting the date format which is present in the extracted date from the file_name (23OCT2023)
    date_obj = datetime.strptime(date_str, date_format) # converting the string extracted from the file_name into datetime object


    df=df[df['SERIES'].isin(['EQ'])]  # filter EQ values
    file_name=date_obj.strftime("%Y%m%d")+".csv" #setting the file name in YYYYMMDD.csv format
    DATASET_CLEANED_FILE_NAME=DATASET_CLEANED_PATH / file_name # creating path to store the cleaned file
    df=df.dropna(how='all', axis='columns') # deleting any column which has been created by mistake having only null values
    df.to_csv(DATASET_CLEANED_FILE_NAME,index=False) # saving the dataframe in csv format 

Location: dataset\cm09NOV2023bhav.csv
File Name: cm09NOV2023bhav.csv
Location: dataset\cm10NOV2023bhav.csv
File Name: cm10NOV2023bhav.csv
Location: dataset\cm13NOV2023bhav.csv
File Name: cm13NOV2023bhav.csv
Location: dataset\cm15NOV2023bhav.csv
File Name: cm15NOV2023bhav.csv
Location: dataset\cm16NOV2023bhav.csv
File Name: cm16NOV2023bhav.csv


# Task 3 : Data Storage

In [92]:
! pip install mysql-connector-python


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [93]:
! pip install SQLAlchemy


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [94]:
! pip install mysqlclient


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [95]:
DB_USER='root'
DB_PASSWORD='Shweta1239#'
DB_HOST='127.0.0.1'
DB_NAME='analytics'
TABLE_EQ='tbl_eq_data'
TABLE_EQ_ANALYTICS='tbl_eq_analytics'

In [96]:
import mysql.connector 

# function to execute sql file
def execute_sql_file(DB_CREATION_FILE_PATH):
    # establish a connection to the database 
    cnx = mysql.connector.connect(user=DB_USER, password=DB_PASSWORD,host=DB_HOST)# database='your_database')
    
    #cnx = create_engine("mysql+mysqldb://{user}:{pw}@{host}/{db}".format(host="localhost", db="analytics", user="root", pw="root"))

    # create a cursor object 
    cursor = cnx.cursor() 

    # read the contents of the .sql file 
    with open(DB_CREATION_FILE_PATH, 'r') as file: 
        sql_script = file.read() 

    # execute the script using the cursor object 
    for result in cursor.execute(sql_script, multi=True): 
        pass 

    # commit the changes to the database 
    cnx.commit() 

    # close the cursor and connection 
    cursor.close() 
    cnx.close() 

In [97]:
DB_CREATION_FILE_PATH=DB_SCRIPTS_PATH / "create_analytics_database.sql"

# Creating a database if it doesnt exist
execute_sql_file(DB_CREATION_FILE_PATH)

In [98]:
TABLE_CREATION_FILE_PATH=DB_SCRIPTS_PATH / "create_eq_table.sql"

# Create table schema if it doesnt exist
execute_sql_file(TABLE_CREATION_FILE_PATH)

In [99]:
from sqlalchemy import create_engine
# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+mysqldb://{user}:{pw}@{host}/{db}".format(host=DB_HOST, db=DB_NAME, user=DB_USER, pw=DB_PASSWORD))

In [100]:
#cnx = mysql.connector.connect(user='root', password='root',host='127.0.0.1')# database='your_database') 

csv_files = glob.glob(os.path.join(DATASET_CLEANED_PATH, "*.csv")) 

# Load all the csv files into data frame and append the data frames to 'df' list
df=[]
for f in csv_files: 
    # read the csv file 
    df.append(pd.read_csv(f))

# concatenate all the data frames into a single data frame using concat() function of pandas
df_result=pd.concat(df,axis=0)

#print(df_result.columns)
#df_result.head()

# Write the data frame into 'tbl_eq_data' table.If the table exists, then replace and create new table
df_result.to_sql(con=engine, name=TABLE_EQ, if_exists='replace')

9017

# Task 4 : Main Process

In [101]:
TABLE_CREATION_FILE_PATH=DB_SCRIPTS_PATH / "create_eq_analytics_table.sql"

In [102]:
# Create table schema if it doesnt exist
execute_sql_file(TABLE_CREATION_FILE_PATH)

In [103]:
df=[]

# Load all the csv files into data frame and append the data frames to 'df' list
for f in csv_files: 
    # read the csv file 
    df.append(pd.read_csv(f))

# concatenate all the data frames into a single data frame using concat() function of pandas
df_result=pd.concat(df,axis=0)
    
df_result=df_result[['SYMBOL','TIMESTAMP','CLOSE','TOTTRDQTY']]

In [104]:
df_result.head()

,SYMBOL,TIMESTAMP,CLOSE,TOTTRDQTY
0,21STCENMGM,09-NOV-2023,18.35,13735
1,360ONE,09-NOV-2023,545.65,197801
2,3IINFOLTD,09-NOV-2023,35.95,400245
3,3MINDIA,09-NOV-2023,31726.55,1981
4,3PLAND,09-NOV-2023,29.05,31979


In [105]:
#sort data date and SYMBOL wise
df_result=df_result.sort_values(by=['TIMESTAMP','SYMBOL'],ascending=[False,True])

In [106]:
df_result.shape

(9017, 4)

In [107]:
# calculate moving average for the last 2 days
df_result['moving_average']=df_result['CLOSE'].rolling(2).mean()

In [108]:
# Write the data frame into 'tbl_eq_analytics' table.If the table exists, then replace and create new table
df_result.to_sql(con=engine, name=TABLE_EQ_ANALYTICS, if_exists='replace') 

9017

# Task 5 : Output

In [109]:
df_result_2=df_result[df_result['CLOSE'] > df_result['moving_average']] # Filter the rows having CLOSE > moving_average

In [110]:
df_result_2.head()

,SYMBOL,TIMESTAMP,CLOSE,TOTTRDQTY,moving_average
1,360ONE,16-NOV-2023,543.10,1136970,281.600
3,3MINDIA,16-NOV-2023,30706.15,2816,15371.525
5,5PAISA,16-NOV-2023,451.95,167205,240.200
9,AARON,16-NOV-2023,263.50,12416,135.275
10,AARTIDRUGS,16-NOV-2023,473.65,164947,368.575


In [111]:
OUTPUT_PATH = Path("output")
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
OUTPUT_FILE_NAME=OUTPUT_PATH/ "output.csv"

In [112]:
df_result_2.to_csv(OUTPUT_FILE_NAME,index=False)